In [38]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten

In [14]:
df = pd.read_excel('Data1.xlsx')
df_or = pd.read_excel('Data1.xlsx')
df = df.drop(["No."],axis = 1)
display(df)

,Tradevalue,Interest,CPI,GDPTW,GDPID,Exchangerate_TW,Exchangerate_ID
0,1043.44,4.543333,81.240000,81139,135355.256125,30.74,7483.333333
1,1254.46,4.670000,81.673333,81801,119275.774773,30.68,8443.333333
2,1236.55,4.750000,81.866667,83514,118987.521626,31.07,8670.000000
3,1214.16,4.710000,82.910000,84271,111509.035986,32.45,9517.500000
4,948.45,4.380000,81.713333,77571,106490.601314,32.53,9895.000000
...,...,...,...,...,...,...,...
91,3287.84,1.670000,103.723333,184142,190205.871091,31.37,15670.000000
92,2631.34,1.793333,104.363333,180361,199728.169480,30.40,15105.000000
93,2623.53,1.880000,105.103333,185510,204896.275645,30.71,14915.333333
94,2743.45,1.880000,105.876667,191900,202499.100288,31.69,15282.666667


In [15]:
df["Tradevalue"] = df["Tradevalue"].shift(-1)
df.dropna(inplace=True)

In [18]:
X = df.drop("Tradevalue",axis=1)
y = df["Tradevalue"]

In [17]:
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(df[feature.tolist() + [target]])

In [11]:
# Function to create sequences
# def create_sequences(df, time_steps=30):
#     X, y = [], []
#     for i in range(len(df) - time_steps):
#         X.append(df[i:i + time_steps, :-1])
#         y.append(df[i + time_steps, -1])
#     return np.array(X), np.array(y)

In [12]:
# sequence_length = 30
# X, y = create_sequences(scaled_data, time_steps=sequence_length)

In [19]:
train_size = int(len(df) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [27]:
model = Sequential([
    LSTM(50, activation='sigmoid', input_shape=(1,6,)),
    Dense(1)
])

In [28]:
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_test, y_test), verbose=1)

Epoch 1/100


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 6), dtype=float32). Expected shape (None, 1, 6), but input has incompatible shape (None, 6)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 6), dtype=float32)
  • training=True
  • mask=None

In [16]:
# Forecasting the next 10 periods
last_sequence = X_test[-1]
predictions = []

In [17]:
for _ in range(10):
    pred = model.predict(last_sequence[np.newaxis, :, :])
    predictions.append(pred[0, 0])
    last_sequence = np.vstack((last_sequence[1:], np.append(pred, last_sequence[0, :-1])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


In [18]:
# Inverse transform the predictions
predictions_rescaled = scaler.inverse_transform(
    np.hstack((np.tile(last_sequence[0, :-1], (10, 1)), np.array(predictions).reshape(-1, 1)))
)[:, -1]

ValueError: operands could not be broadcast together with shapes (10,6) (7,) (10,6) 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Flatten, Dense

# Load and preprocess the data
df = pd.read_excel('Data1.xlsx')
df = df.drop(["No."], axis=1)

# Shift the target variable for next-step prediction
df["Tradevalue"] = df["Tradevalue"].shift(-1)
df.dropna(inplace=True)

# Separate features and target
X = df.drop("Tradevalue", axis=1).values
y = df["Tradevalue"].values

# Normalize the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Define time_steps
time_steps = 3  # Using more than 1 to capture temporal dependencies

# Function to create sequences
def create_sequences(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Create sequences
X_sequences, y_sequences = create_sequences(X_scaled, y, time_steps)

# Train-test split
train_size = int(len(X_sequences) * 0.9)
X_train, X_test = X_sequences[:train_size], X_sequences[train_size:]
y_train, y_test = y_sequences[:train_size], y_sequences[train_size:]

# Define the model
model = Sequential()

# Conv1D layers require input sequences longer than the kernel_size
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))

# LSTM layers
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.3))

# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model with the correct loss function for regression
model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
model.fit(X_train, y_train, epochs=3000, batch_size=8, validation_data=(X_test, y_test), verbose=1)


Epoch 1/3000


C:\Users\bsean\anaconda3\envs\bujanx\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 2109.3833 - val_loss: 3129.2280
Epoch 2/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2112.6143 - val_loss: 3126.9595
Epoch 3/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2046.4476 - val_loss: 3086.0137
Epoch 4/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1963.3447 - val_loss: 2617.2231
Epoch 5/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1496.8672 - val_loss: 733.0213
Epoch 6/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 720.7364 - val_loss: 756.9825
Epoch 7/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 718.2788 - val_loss: 797.2285
Epoch 8/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 681.2921 - val_loss: 755.4171
Epoch 9/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 604.8744 - val_loss: 792.0629
Epoch 10/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 587.0023 - val_loss: 768.0876
Epoch 11/3000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 613.9355 - val_loss: 786.0291
Epoch 12/30

In [ ]:
# Inference Function
def make_prediction(user_input):
    """
    Predict the next value based on user input.

    Parameters:
        user_input (array-like): An array of 6 values (independent variables).

    Returns:
        float: Predicted next "Tradevalue".
    """
    # Scale and reshape user input
    user_input_scaled = scaler.transform([user_input])
    user_input_reshaped = user_input_scaled.reshape((1, time_steps, X_train.shape[2]))

    # Predict and return the result
    prediction_scaled = model.predict(user_input_reshaped)
    prediction_rescaled = scaler.inverse_transform(
        np.hstack((user_input_scaled, [[prediction_scaled[0, 0]]]))
    )[:, -1]

    return prediction_rescaled[0]

# Example Usage of Inference
# Assuming user inputs are similar to the feature columns
example_user_input = X.iloc[-1].values  # Example: the last row of the features
predicted_tradevalue = make_prediction(example_user_input)

print("Predicted next Tradevalue:", predicted_tradevalue)